The titles of documents harbor valuable information beyond their themes. By incorporating titles, we have the potential to enrich the context of our analysis. Additionally, I plan to utilize a compact pretrained Word2Vec algorithm for this purpose.

In [ ]:
from gensim.models import KeyedVectors


path_to_word2vec_model = 'frWiki_no_lem_no_postag_no_phrase_1000_skip_cut100.bin'
word2vec_model = KeyedVectors.load_word2vec_format(path_to_word2vec_model, binary=True)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
def remove_stop_words(sentence):

    cleaned_text = re.sub(r'[^\w\s]|l\'', '', sentence)

    words = word_tokenize(cleaned_text)

    filtered_words = [word.lower() for word in words if word.lower() not in stopwords.words('french')]

    return filtered_words

In [ ]:
remove_stop_words(sessions[0][0][1]["title"])

In [ ]:
def get_document_words(document):

    representation = [document[0].lower()]

    representation += remove_stop_words(document[1]["title"])

    return representation

In [ ]:
def get_session_words(session):

    result = []

    for document in session:

        result += get_document_words(document)

    return set(result)

In [ ]:
def get_session_representation(session, model):

    session_words = get_session_words(session)

    return np.array([model[word] for word in session_words if word in model.key_to_index])

In [ ]:
import kmapper as km
from kmapper.jupyter import display
from sklearn.manifold import Isomap

In [ ]:
mapper  = km.KeplerMapper(verbose=1)

projected_data = mapper.fit_transform(test, projection=[Isomap(n_components=300, n_jobs=1), UMAP(n_components=2, random_state=42)])

G = mapper.map(projected_data, test, clusterer=DBSCAN(metric="cosine", eps=1))